## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,55.78,89,0,9.82,clear sky
1,1,Lagoa,PT,39.0500,-27.9833,59.50,69,51,12.35,broken clouds
2,2,Port Hardy,CA,50.6996,-127.4199,46.58,71,75,9.22,broken clouds
3,3,Whitehorse,CA,60.7161,-135.0538,26.10,49,75,0.00,broken clouds
4,4,Jamestown,US,42.0970,-79.2353,35.24,74,100,23.02,snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                    (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             204
City                   204
Country                201
Lat                    204
Lng                    204
Max Temp               204
Humidity               204
Cloudiness             204
Wind Speed             204
Current Description    204
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

Unnamed: 0             201
City                   201
Country                201
Lat                    201
Lng                    201
Max Temp               201
Humidity               201
Cloudiness             201
Wind Speed             201
Current Description    201
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Hilo,US,80.28,scattered clouds,19.7297,-155.0900,
8,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,
9,Rikitea,PF,77.72,clear sky,-23.1203,-134.9692,
12,Omboue,GA,76.21,light rain,-1.5746,9.2618,
19,Meulaboh,ID,75.29,light rain,4.1363,96.1285,
21,Bubaque,GW,77.27,few clouds,11.2833,-15.8333,
29,Pontes E Lacerda,BR,88.70,clear sky,-15.2261,-59.3353,
32,Mabaruma,GY,87.01,broken clouds,8.2000,-59.7833,
35,Atuona,PF,79.86,scattered clouds,-9.8000,-139.0333,
37,Bahia Honda,CU,82.69,scattered clouds,22.9036,-83.1592,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] =f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Hilo,US,80.28,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
8,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
9,Rikitea,PF,77.72,clear sky,-23.1203,-134.9692,People ThankYou
12,Omboue,GA,76.21,light rain,-1.5746,9.2618,Hotel Olako
19,Meulaboh,ID,75.29,light rain,4.1363,96.1285,Kost Pria
...,...,...,...,...,...,...,...
656,Cururupu,BR,84.40,overcast clouds,-1.8283,-44.8683,POUSADA TRAVÉS'CIA
658,Golden,US,77.22,broken clouds,39.7555,-105.2211,"The Golden Hotel, Ascend Hotel Collection"
659,Vallenar,CL,76.39,broken clouds,-28.5708,-70.7581,Humacao Bed & Breakfast
663,Daloa,CI,83.01,overcast clouds,6.8774,-6.4502,Imans Hotel Daloa


In [10]:
nan_value=float("NaN")
hotel_df = hotel_df.replace("", nan_value)

In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df=hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Hilo,US,80.28,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
8,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
9,Rikitea,PF,77.72,clear sky,-23.1203,-134.9692,People ThankYou
12,Omboue,GA,76.21,light rain,-1.5746,9.2618,Hotel Olako
19,Meulaboh,ID,75.29,light rain,4.1363,96.1285,Kost Pria
...,...,...,...,...,...,...,...
654,Le Port,RE,78.80,broken clouds,-20.9373,55.2919,KORBEY D'OR OUEST
656,Cururupu,BR,84.40,overcast clouds,-1.8283,-44.8683,POUSADA TRAVÉS'CIA
658,Golden,US,77.22,broken clouds,39.7555,-105.2211,"The Golden Hotel, Ascend Hotel Collection"
659,Vallenar,CL,76.39,broken clouds,-28.5708,-70.7581,Humacao Bed & Breakfast


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [2]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

NameError: name 'locations' is not defined